In [1]:
# -*- coding=utf-8 -*-
import pymysql
import sys
pymysql.install_as_MySQLdb()
import MySQLdb as mdb
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from itertools import chain
import warnings
import os

warnings.filterwarnings('ignore')

default_config_115 = {
    'host': '175.25.50.115',
    'port': 3306,
    'user': 'xtech',
    'passwd': 'x-tech-123',
    'db': 'jydb',
    'charset': 'gbk'
}

xtech_config = {
    'host': '175.25.50.115',
    'port': 3306,
    'user': 'xtech',
    'passwd': 'x-tech-123',
    'db': 'xtech',
    'charset': 'gbk',
}


class FetchData(object):
    def __init__(self, config):
        self.config = config
        # self.conn = mdb.connect(**config)
        # self.conn.autocommit(1)
        self.conn = None

    def __del__(self):
        if self.conn is not None:
            self.conn.close()

    def _exec(self, query, describe=False):
        if self.conn is None:
            self.conn = mdb.connect(**self.config)
            self.conn.autocommit(1)
        try:
            cursor = self.conn.cursor()
            cursor.execute(query)
            cursor.scroll(0, mode='absolute')
            results = cursor.fetchall()
            description = cursor.description
        except:
            # print(query, ' error')
            # import traceback
            # traceback.print_exc()
            self.conn.rollback()
            cursor.close()
            return None, None
        cursor.close()
        if not describe:
            return results
        else:
            columns = []
            for x in description:
                columns.append(x[0])
            return columns, results

    def fetch_tables(self, verbose=False):
        tables = []
        query = 'SHOW TABLES'
        results = self._exec(query)

        for x in results:
            tables.append(x[0])
        if verbose is True:
            print(results)
        return tables

    def fetch_columns(self, table_name, verbose=False):
        columns = []
        query = 'SHOW COLUMNS FROM %s' % table_name
        results = self._exec(query)
        for x in results:
            columns.append(x[0])
        if verbose is True:
            print(results)
        return columns

    def _fetch_index(self, table_name, verbose=False):
        ids = []
        query = 'SHOW INDEX FROM %s' % table_name
        results = self._exec(query)
        for x in results:
            ids.append(x[0])
        if verbose is True:
            print(results)
        return ids

    def ping(self, table_name, verbose=False):
        query = 'SELECT * FROM %s LIMIT %s' % (table_name, 1)
        columns, result = self._exec(query)
        if verbose is True:
            print(result[0])
        if result is None:
            print('Query failed!')
        else:
            print('Query success!')
        return result

    def head(self, table_name, limit=5):
        query = 'SELECT * FROM %s  ORDER BY id ASC LIMIT %s' % (table_name, limit)
        columns, results = self._exec(query, True)
        # columns = self.fetch_columns(table_name)
        df_head = pd.DataFrame(np.array(results), columns=columns)
        print(df_head)
        return df_head

    def tail(self, table_name, limit=5):
        query = 'SELECT * FROM %s ORDER BY id DESC LIMIT %s' % (table_name, limit)
        columns, results = self._exec(query, True)
        # columns = self.fetch_columns(table_name)
        df_tail = pd.DataFrame(np.array(results), columns=columns)
        # print (df_tail)
        return df_tail

    def getdata(self, table_name, columns=None, date_name=None, start_date=None, end_date=None):
        if isinstance(columns, list):
            columns = ",".join(columns)
        else:
            columns = '*'

        if date_name is None:
            query = 'SELECT %s FROM %s' % (columns, table_name)
        else:
            start_date = '"' + start_date + '"'
            end_date = '"' + end_date + '"'
            query = 'SELECT %s FROM %s  where %s >= %s and %s <= %s order by %s asc;' % (
            columns, table_name, date_name, start_date, date_name, end_date, date_name)

        columns, results = self._exec(query)
        # print(results)
        if results is None:
            print('Query Failed!')
            df = None
        else:
            df = pd.DataFrame(np.array(results), columns=columns)
        return df

    def get_minmax(self, table_name, columns):
        code = self.getdata(table_name, columns)
        print(code.min(), code.max())
        return (code.min(), code.max())

    def get_data(self, table_name):
        query = 'SELECT * FROM {}'.format(table_name)
        columns, results = self._exec(query, True)
        if results is None:
            # print("Query Failed!")
            df = None
        else:
            df = pd.DataFrame(np.array(results), columns=columns)
            df.index = np.arange(df.shape[0])
        return df

    def get_day_data(self, table_name, date_name, date):
        query = 'SELECT * FROM {} where {} = "{}" '.format(table_name, date_name, date)
        columns, results = self._exec(query, True)
        if results is None:
            print('Query Failed!')
            df = None
        else:
            # df_col = self.fetch_columns(table_name)
            df = pd.DataFrame(np.array(results), columns=columns)
            df.index = np.arange(df.shape[0])
        return df

    def get_quarter_data(self, table_name, date_name, quarter_time, limit):
        query = 'SELECT * FROM %s where %s <= "%s" ORDER BY %s DESC limit %s' % (
        table_name, date_name, quarter_time, date_name, limit)
        columns, results = self._exec(query, True)
        if results is None:
            print('Query Failed!')
            df = None
        else:
            # df_col = self.fetch_columns(table_name)
            df = pd.DataFrame(np.array(results), columns=columns)
            df.index = np.arange(df.shape[0])
        return df

    def exec_sql(self, query):
        columns, results = self._exec(query, True)
        if results is None:
            return None
        else:
            df = pd.DataFrame(np.array(results), columns=columns)
            return df

    def exec_file(self, filename):
        with open(filename, 'r') as f:
            query = f.read()
            columns, results = self._exec(query, True)
            df = pd.DataFrame(np.array(results), columns=columns)
        return df


class Wapper_FetchData(FetchData):
    def __init__(self, config, SecuCategory=1):
        super(Wapper_FetchData, self).__init__(config=config)
        self.SecuCategory = SecuCategory

    def set_SecuCategory(self, SecuCategory):
        self.SecuCategory = SecuCategory
        print("Now SecuCategory:", self.SecuCategory)

    def decorator(func):
        def wrapper(self, *args, **kw):
            describe = True
            if not describe:
                return func(*args, **kw)
            else:
                columns, results = func(self, *args, **kw)
                if results is None:
                    return columns, results
                elif ("SecuCode" in columns):
                    return columns, results
                elif ("InnerCode" in columns):
                    query = 'SELECT InnerCode,SecuCode FROM SecuMain where SecuCategory = %s and (SecuMarket = 83 or SecuMarket = 90);' % (
                        self.SecuCategory)
                    df_main = self.exec_sql(query)
                    df_main = df_main[['InnerCode', 'SecuCode']]
                    df_main['InnerCode'] = df_main['InnerCode'].astype(np.int64)
                    df = pd.DataFrame(np.array(results), columns=columns)
                    df['InnerCode'] = df['InnerCode'].astype(np.int64)
                    out_df = pd.merge(df_main, df, on="InnerCode")
                    return out_df.columns, out_df.values
                elif ("CompanyCode" in columns):
                    query = 'SELECT CompanyCode,SecuCode FROM SecuMain where SecuCategory = %s and (SecuMarket = 83 or SecuMarket = 90);' % (
                        self.SecuCategory)
                    df_main = self.exec_sql(query)
                    df_main = df_main[['CompanyCode', 'SecuCode']]
                    df_main['CompanyCode'] = df_main['CompanyCode'].astype(np.int64)
                    df = pd.DataFrame(np.array(results), columns=columns)
                    df['CompanyCode'] = df['CompanyCode'].astype(np.int64)
                    out_df = pd.merge(df_main, df, on="CompanyCode")
                    columns = out_df.columns
                    results = out_df.values
                    return out_df.columns, out_df.values
                else:
                    return columns, results

        return wrapper

    @decorator
    def _exec(self, query, describe=True):
        if self.conn is None:
            self.conn = mdb.connect(**self.config)
            self.conn.autocommit(1)

        try:
            cursor = self.conn.cursor()
            cursor.execute(query)
            cursor.scroll(0, mode='absolute')
            results = cursor.fetchall()
            description = cursor.description
        except:
            # print(query, ' error')
            # import traceback
            # traceback.print_exc()
            self.conn.rollback()
            results = None
            description = None
            return None, None
        cursor.close()
        if not describe:
            return results
        else:
            columns = []
            for x in description:
                columns.append(x[0])
            return columns, results

    def get_IndexComponent(self, SecuCode, Date):
        '''
            SecuCode type:str  exmple: '000300' (hs300)
            Date: type:str. exmple '2017-04-10'
        '''
        query = 'SELECT InnerCode, SecuCode FROM SecuMain where SecuCode = {} and SecuCategory = 4 and (SecuMarket = 83 or SecuMarket = 90);'.format(
            SecuCode)
        df_main = self.exec_sql(query)
        # print(df_main)
        assert len(df_main) == 1
        IndexCode = int(df_main['InnerCode'].values)
        Date = pd.to_datetime(Date)
        query2 = 'select InnerCode from SA_TradableShare where IndexCode = {} and EndDate = "{}" '.format(IndexCode,
                                                                                                          Date)
        df = self.exec_sql(query2)
        result = pd.DataFrame(df['SecuCode'].unique(), columns=[SecuCode])
        return result

    def get_IndexComponent2(self, SecuCode, Date=None):
        '''
            SecuCode type:str  exmple: '000300' (hs300)
            Date: type:str. exmple '2017-04-10'
        '''
        query = 'SELECT InnerCode, SecuCode FROM SecuMain where SecuCode = {} and SecuCategory = 4 and (SecuMarket = 83 or SecuMarket = 90);'.format(
            SecuCode)
        df_main = self.exec_sql(query)
        # print(df_main)
        assert len(df_main) == 1
        IndexCode = int(df_main['InnerCode'].values)
        Date = pd.to_datetime(Date)
        query2 = 'select SecuInnerCode as InnerCode from LC_IndexComponent where IndexInnerCode = {} and OutDate is Null'.format(
            IndexCode)
        df = self.exec_sql(query2)
        result = pd.DataFrame(df['SecuCode'].unique(), columns=[SecuCode])
        return result


class XTechData(FetchData):
    def __init__(self, config):
        super(XTechData, self).__init__(config=config)
        DB_URI = 'mysql+mysqldb://{}:{}@{}:{}/{}?charset=utf8'.format(config['user'],
                                                                      config['passwd'],
                                                                      config['host'],
                                                                      config['port'],
                                                                      config['db'])
        self.engine = create_engine(DB_URI, encoding='utf-8')

    def insert(self, df, date, mode='append'):
        with self.engine.connect() as conn:
            df.to_sql(date, conn, if_exists=mode)


In [2]:
import baostock as bs
def find_index(date):
    lg = bs.login()
    rs = bs.query_zz500_stocks(date)
    _stocks = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        _stocks.append(rs.get_row_data())
    a = list(pd.DataFrame(_stocks)[1])
    symbol_list =  [ _[3:] for _ in a]
    bs.logout()
    return symbol_list

In [3]:
def get_all_data():
    jydb_loader = FetchData(config=default_config_115)
    secu_loader = Wapper_FetchData(config=default_config_115)
    xtech_db = XTechData(config=xtech_config)
    all_date = xtech_db.fetch_tables()
    all_date = [i for i in all_date if i[:4]=='2017' or i[:4]=='2018' or i[:4]=='2016' or i[:4] == '2015']
    fetch = xtech_db.get_data("`{}`".format(all_date[0])).sort_values('SecuCode')
    all_data = pd.DataFrame()
    print('fetching data from server....')
    for i in range(len(all_date) - 1):
        p = str(i+1)+'/'+str(len(all_date)-1)+'    '+str(all_date[i+1])
#         print (p, end="\r")
        print(p)
        next_fetch = xtech_db.get_data("`{}`".format(all_date[i+1])).sort_values('SecuCode')
        fetch["NextReturn"] = 100 * (next_fetch['ClosePrice'].astype('float32') - next_fetch['PrevClosePrice'].astype('float32'))/next_fetch['PrevClosePrice'].astype('float32')
#         print(fetch["SecuCode"] in find_index(str(all_date[i])))
        stkpool = find_index(str(all_date[i]))
        stock_pool = [_ in stkpool for _ in fetch["SecuCode"]]
        print(len(fetch[stock_pool]))
        all_data = all_data.append(fetch[stock_pool])
        fetch = next_fetch
    print(all_data.columns)
    all_data['TradingDay'] = all_data['TradingDay'].astype('datetime64')
    #deal with nan
    all_data['NextReturn'] = np.nan_to_num(all_data['NextReturn'].values)
    return all_data    

In [4]:
len(find_index(str(all_date[-1])))

NameError: name 'all_date' is not defined

In [4]:
all_data = get_all_data()

fetching data from server....
1/974    2015-01-06
login success!
logout success!
500
2/974    2015-01-07
login success!
logout success!
500
3/974    2015-01-08
login success!
logout success!
500
4/974    2015-01-09
login success!
logout success!
500
5/974    2015-01-12
login success!
logout success!
500
6/974    2015-01-13
login success!
logout success!
500
7/974    2015-01-14
login success!
logout success!
500
8/974    2015-01-15
login success!
logout success!
500
9/974    2015-01-16
login success!
logout success!
500
10/974    2015-01-19
login success!
logout success!
500
11/974    2015-01-20
login success!
logout success!
500
12/974    2015-01-21
login success!
logout success!
500
13/974    2015-01-22
login success!
logout success!
500
14/974    2015-01-23
login success!
logout success!
500
15/974    2015-01-26
login success!
logout success!
500
16/974    2015-01-27
login success!
logout success!
500
17/974    2015-01-28
login success!
logout success!
500
18/974    2015-01-29
login 

login success!
logout success!
500
147/974    2015-08-10
login success!
logout success!
500
148/974    2015-08-11
login success!
logout success!
500
149/974    2015-08-12
login success!
logout success!
500
150/974    2015-08-13
login success!
logout success!
500
151/974    2015-08-14
login success!
logout success!
500
152/974    2015-08-17
login success!
logout success!
500
153/974    2015-08-18
login success!
logout success!
500
154/974    2015-08-19
login success!
logout success!
500
155/974    2015-08-20
login success!
logout success!
500
156/974    2015-08-21
login success!
logout success!
500
157/974    2015-08-24
login success!
logout success!
500
158/974    2015-08-25
login success!
logout success!
500
159/974    2015-08-26
login success!
logout success!
500
160/974    2015-08-27
login success!
logout success!
500
161/974    2015-08-28
login success!
logout success!
500
162/974    2015-08-31
login success!
logout success!
500
163/974    2015-09-01
login success!
logout success!


login success!
logout success!
500
291/974    2016-03-16
login success!
logout success!
500
292/974    2016-03-17
login success!
logout success!
500
293/974    2016-03-18
login success!
logout success!
500
294/974    2016-03-21
login success!
logout success!
500
295/974    2016-03-22
login success!
logout success!
500
296/974    2016-03-23
login success!
logout success!
500
297/974    2016-03-24
login success!
logout success!
500
298/974    2016-03-25
login success!
logout success!
500
299/974    2016-03-28
login success!
logout success!
500
300/974    2016-03-29
login success!
logout success!
500
301/974    2016-03-30
login success!
logout success!
500
302/974    2016-03-31
login success!
logout success!
500
303/974    2016-04-01
login success!
logout success!
500
304/974    2016-04-05
login success!
logout success!
500
305/974    2016-04-06
login success!
logout success!
500
306/974    2016-04-07
login success!
logout success!
500
307/974    2016-04-08
login success!
logout success!


login success!
logout success!
500
435/974    2016-10-19
login success!
logout success!
500
436/974    2016-10-20
login success!
logout success!
500
437/974    2016-10-21
login success!
logout success!
500
438/974    2016-10-24
login success!
logout success!
500
439/974    2016-10-25
login success!
logout success!
500
440/974    2016-10-26
login success!
logout success!
500
441/974    2016-10-27
login success!
logout success!
500
442/974    2016-10-28
login success!
logout success!
500
443/974    2016-10-31
login success!
logout success!
500
444/974    2016-11-01
login success!
logout success!
500
445/974    2016-11-02
login success!
logout success!
500
446/974    2016-11-03
login success!
logout success!
500
447/974    2016-11-04
login success!
logout success!
500
448/974    2016-11-07
login success!
logout success!
500
449/974    2016-11-08
login success!
logout success!
500
450/974    2016-11-09
login success!
logout success!
500
451/974    2016-11-10
login success!
logout success!


login success!
logout success!
500
579/974    2017-05-22
login success!
logout success!
500
580/974    2017-05-23
login success!
logout success!
500
581/974    2017-05-24
login success!
logout success!
500
582/974    2017-05-25
login success!
logout success!
500
583/974    2017-05-26
login success!
logout success!
500
584/974    2017-05-31
login success!
logout success!
500
585/974    2017-06-01
login success!
logout success!
500
586/974    2017-06-02
login success!
logout success!
500
587/974    2017-06-05
login success!
logout success!
500
588/974    2017-06-06
login success!
logout success!
500
589/974    2017-06-07
login success!
logout success!
500
590/974    2017-06-08
login success!
logout success!
500
591/974    2017-06-09
login success!
logout success!
500
592/974    2017-06-12
login success!
logout success!
500
593/974    2017-06-13
login success!
logout success!
500
594/974    2017-06-14
login success!
logout success!
500
595/974    2017-06-15
login success!
logout success!


login success!
logout success!
500
723/974    2017-12-19
login success!
logout success!
500
724/974    2017-12-20
login success!
logout success!
500
725/974    2017-12-21
login success!
logout success!
500
726/974    2017-12-22
login success!
logout success!
500
727/974    2017-12-25
login success!
logout success!
500
728/974    2017-12-26
login success!
logout success!
500
729/974    2017-12-27
login success!
logout success!
500
730/974    2017-12-28
login success!
logout success!
500
731/974    2017-12-29
login success!
logout success!
500
732/974    2018-01-02
login success!
logout success!
500
733/974    2018-01-03
login success!
logout success!
500
734/974    2018-01-04
login success!
logout success!
500
735/974    2018-01-05
login success!
logout success!
500
736/974    2018-01-08
login success!
logout success!
500
737/974    2018-01-09
login success!
logout success!
500
738/974    2018-01-10
login success!
logout success!
500
739/974    2018-01-11
login success!
logout success!


login success!
logout success!
500
867/974    2018-07-24
login success!
logout success!
500
868/974    2018-07-25
login success!
logout success!
500
869/974    2018-07-26
login success!
logout success!
500
870/974    2018-07-27
login success!
logout success!
500
871/974    2018-07-30
login success!
logout success!
500
872/974    2018-07-31
login success!
logout success!
500
873/974    2018-08-01
login success!
logout success!
500
874/974    2018-08-02
login success!
logout success!
500
875/974    2018-08-03
login success!
logout success!
500
876/974    2018-08-06
login success!
logout success!
500
877/974    2018-08-07
login success!
logout success!
500
878/974    2018-08-08
login success!
logout success!
500
879/974    2018-08-09
login success!
logout success!
500
880/974    2018-08-10
login success!
logout success!
500
881/974    2018-08-13
login success!
logout success!
500
882/974    2018-08-14
login success!
logout success!
500
883/974    2018-08-15
login success!
logout success!


In [5]:
all_data.reset_index(drop=True, inplace=True)

In [6]:
file_list = os.listdir("data_backup/alpha")
factors = []
for alpha in file_list:
    factors.append(pd.read_csv('data_backup/alpha/'+alpha)[1:])
factor_index = 0
for factor in factors:
    factor_name = 'factor'+str(factor_index)
    p = "reading " + factor_name + '/' + str(len(factors)-1)
    print(p)
    factor_index += 1
    new_column = {}
    for i in list(factor.columns):
        if 'a' not in i:
            new_column[i] = (6-len(i))*'0'+i
    factor = factor.rename(index=str, columns=new_column)
    factor = factor.sort_index(1)
    all_factors = []
    for i in range(factor.shape[0]):
        p = str(i) + "/" + str(factor.shape[0])
        print(p,end='\r')
        if factor.iloc[i,-5][:4] != '2017' and factor.iloc[i,-5][:4] != '2018' and  factor.iloc[i,-5][:4] != '2016' and factor.iloc[i,-5][:4] != '2015':
            continue
        new_factor = factor.iloc[i,:-7].T.to_frame()
        new_factor['TradingDay'] = pd.to_datetime(factor.iloc[i,-5])
        new_factor = new_factor.rename(index=str, columns={list(new_factor.columns)[0]: factor_name})
        new_factor['SecuCode'] = new_factor.index
        all_factors.append(new_factor)
    print("concatenating.....")
    all_factors = pd.concat(all_factors)
    all_data = pd.merge(all_data, all_factors,left_on=  ['SecuCode', 'TradingDay'],
               right_on= ['SecuCode','TradingDay'],
               how = 'left')
    

reading factor0/16
concatenating.....
reading factor1/16
concatenating.....
reading factor2/16
concatenating.....
reading factor3/16
concatenating.....
reading factor4/16
concatenating.....
reading factor5/16
concatenating.....
reading factor6/16
concatenating.....
reading factor7/16
concatenating.....
reading factor8/16
concatenating.....
reading factor9/16
concatenating.....
reading factor10/16
concatenating.....
reading factor11/16
concatenating.....
reading factor12/16
concatenating.....
reading factor13/16
concatenating.....
reading factor14/16
concatenating.....
reading factor15/16
concatenating.....
reading factor16/16
concatenating.....


In [7]:
all_data['NextReturn'] = np.nan_to_num(all_data['NextReturn'].values)
a = all_data['NextReturn'].values
low_thred = np.percentile(a,33.3333)
high_thred = np.percentile(a,66.6666)
all_data['NextReturnCate'] = np.where(all_data['NextReturn']<low_thred, -1, np.where(all_data['NextReturn']<high_thred,0,1))

In [8]:
all_data.to_csv("new_all.csv")

In [12]:
all_data[all_data["TradingDay"] == "2015-01-05"]

,index,InnerCode,TradingDay,PrevClosePrice,OpenPrice,HighPrice,LowPrice,ClosePrice,TurnoverVolume,TurnoverValue,...,factor8,factor9,factor10,factor11,factor12,factor13,factor14,factor15,factor16,NextReturnCate
0,4,20,2015-01-05,7.0500,7.1000,7.3900,7.0200,7.0800,107864712,775346443.8200,...,94.8675,57.9422,2.3118,0.378723,397.326,4.99105,18.8138,66.1581,394.355,-1
1,10,38,2015-01-05,8.8900,8.9000,9.2000,8.8500,9.1300,77484386,703147665.3100,...,106.916,19.9196,2.3082,0.464435,177.222,3.01431,21.688,75.025,176.328,0
2,17,57,2015-01-05,7.1800,7.3000,7.5000,7.2100,7.4300,24220257,178564350.8600,...,32.477,14.2153,2.2056,0.553191,105.605,-0.361918,62.8718,45.9379,106.274,1
3,24,68,2015-01-05,47.7300,47.9500,49.2900,47.0100,48.3200,2618894,126230178.3100,...,84.4709,3.83311,3.6776,0.642475,181.83,-0.100233,26.8543,59.7992,180.984,1
4,26,72,2015-01-05,8.4500,8.4600,8.7200,8.4200,8.4500,14874851,126989523.1600,...,83.6455,2.12751,2.5496,0.235521,17.7624,0.260867,17.5198,75.3715,17.7945,1
5,27,74,2015-01-05,8.4600,8.9000,9.3100,8.7200,9.3100,142248249,1305712644.0600,...,109.78,13.8116,2.8214,0.368056,115.851,7.01518,28.2124,73.0431,113.283,1
6,42,93,2015-01-05,30.1200,30.1100,31.1000,29.7000,30.7800,4726339,143523611.8000,...,12.1066,5.06253,8.5167,0.885331,155.042,-8.30517,26.935,11.6664,153.638,1
7,43,94,2015-01-05,19.1600,19.7000,20.1500,19.2000,19.5000,27378808,536589479.8200,...,51.3354,8.14191,2.7087,0.726479,156.63,-8.21533,36.0217,32.5844,155.543,1
8,50,104,2015-01-05,15.1700,15.1700,15.6900,15.0700,15.5800,7650032,118016675.1400,...,26.7033,4.72898,4.4898,0.796537,73.7728,-1.94696,21.7589,20.1875,73.7532,1
9,53,107,2015-01-05,5.7300,5.7900,6.1900,5.7900,6.0500,21890029,131756523.0300,...,54.856,11.2392,3.1405,0.600575,66.7736,-1.86582,-612.465,51.5134,67.0081,0


In [13]:
all_data.to_csv("all_data.csv")

In [50]:
all_data = pd.read_csv("all_data.csv")